In [ ]:
from dependencies.config import Config
#from jobs.etl_jobs import EtlJobs
import dependencies.spark as spark
import dependencies.logging as log
from pyspark import SparkContext, SparkConf, SQLContext
import utils.utils as util
from pyspark.sql.types import *
#import password as p
import os

In [ ]:
server = "SDBUETMSQL06"
database = "PAS:Test"
table = "contrato" #Para probar aqui cambian el nombre de la tabla en cuestion
user = 'X_PAST_TERA_UPS'
password = "Neuquen#Sanmartindelosandes11382"
DRIVERCLASS = "com.microsoft.sqlserver.jdbc.SQLServerDriver"
URL = f'jdbc:sqlserver://{server};databaseName={database}'

In [ ]:
#Iniciar sesion en Spark

conf = SparkConf() \
    .setAppName("test_pas_tsl") \
    .setMaster("yarn") \
    .set("spark.driver.extraClassPath",'mssql-jdbc-11.2.0.jre8.jar')
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

In [ ]:
#Traer tabla mediante jdbc y guardarlo en un df

try:
    print("Generando Sesión con SQL...")
    
    query = f"SELECT * FROM {table}" #Aplicar filtro con WHERE para pruebas, no traer tabla completa.
      
    jdbcDF = spark.read.format("jdbc") \
               .option("url", URL) \
               .option("user", user) \
               .option("password", password) \
               .option("driver", DRIVERCLASS) \
               .option("query", query) \
               .load()      
      
    #jdbcDF.show()
    jdbcDF.printSchema()
        
except ValueError as error:
      logging.info("Fallo conexión del conector JDBC con la DB.", error)

In [ ]:
#Generacion de Path (pueden probarlo con un path local)

path = "/user/se35430a/pas_tsl_test"
path_stg = "raw_data"
name_table = table
path_output_stg = os.path.join(path, name_table.lower(), path_stg, "")
    
print(f'Path sin particionar:{path_output_stg}')

In [ ]:
#Guardado y almacenamiento en formato Parquet (puede modificarse y hacerse a csv entre otros formatos)

try:
    jdbcDF.write.save(path=path_output_stg, format="parquet", mode="overwrite")
    
    
except ValueError as error:
    logging.error("No se pudo escribir en HDFS.", error)
    logging.info('Escritura en HDFS correcta')